### Basic SQL Agent

Trying to make a simple SQL agent. LLMs = good. First step get prompting + sql database + code interpreter working. Then deal with custom business logic. 

In [6]:
import os 
import time
from dotenv import load_dotenv 

import re 
import numpy as np 
import pandas as pd

from sqlalchemy import create_engine
from langchain_openai import ChatOpenAI 
from langchain_community.utilities import SQLDatabase

class Config:
    """SQL agent config"""
    def __init__(
        self,
        env_fpath,
        sql_db_fpath,
        openai_model = 'gpt-4-1106-preview',
        temperature  = 0.7
        ):
        load_dotenv(env_fpath)
        openai_api_key = os.getenv("OPENAI_API_KEY")
        
        self.llm = ChatOpenAI(
            model=openai_model,
            temperature=temperature,
            api_key=openai_api_key
            )
        self.db = SQLDatabase.from_uri(f"sqlite:///{sql_db_fpath}")
        self.engine = create_engine(f"sqlite:///{sql_db_fpath}")

In [2]:
SRC_PATH     = os.path.dirname(os.getcwd())
ENV_FPATH    = f'{SRC_PATH}/keys.env'
SQL_DB_FPATH = f'{SRC_PATH}/synthetic_data.db' 

print(f'src path: {SRC_PATH}')
print(f'env fpath: {ENV_FPATH}')
print(f'sql db fpath: {SQL_DB_FPATH}')

config = Config(
    env_fpath    = ENV_FPATH,
    sql_db_fpath = SQL_DB_FPATH  
    )   

src path: /Users/jcheigh/text2sql/src
env fpath: /Users/jcheigh/text2sql/src/keys.env
sql db fpath: /Users/jcheigh/text2sql/src/synthetic_data.db


### Agent Logic

This agent will take the user query, generate/execute sql to pull the relevant data, and then generate/execute python code to return the desired result. Below will show the system prompts/a labeled user query/desired behavior. The actual system prompts will include this one shot example.

For now, we will stick to the basics, both in terms of user query complexity and agent complexity:
- Users will ask basic data analytics about relevant tables, with very clear instructions
- Our system will have essentially no error handling 
- Our system will have no CoT or an advanced cognitive architecture. 

*User Query:* Calculate the correlation between 7 year treasury yields and stocks close over the last 30 days in the table. 
**Step 1- SQL Query**

*System Prompt:* 
    
Given a user query, generate a syntatically correct SQLite query wrapped in ```sql {QUERY}``` that pulls all the data relevant to the query. Do not perform any computation, assume that the user is able to take the raw data and perform the relevant computations to reach the desired result. Be mindful of how many days of data to pull, as certain queries may specific n days but require more than n to compute the result. Make sure the column names in the resulting table are clear. Here is the database schema: {schema_context} 

*Labeled Answer:*
``` sql 
SELECT 
    y.date AS date,
    y.yield_7_year AS treasury_yield_7_year,
    o.close AS stock_close
FROM 
    treasury_yields y
INNER JOIN 
    ohlc o ON y.date = o.date
ORDER BY 
    y.date DESC
LIMIT 30;
```

*Result of Executing SQL:*

| Date                | Treasury Yield (7-Year) | Stock Close   |
|---------------------|--------------------------|---------------|
| 2024-01-01 00:00:00 | 4.113933                | 84.676268     |
| 2023-12-29 00:00:00 | 4.117221                | 100.393128    |
| 2023-12-28 00:00:00 | 2.391113                | 112.975980    |
| 2023-12-27 00:00:00 | 1.482054                | 119.224503    |
| 2023-12-26 00:00:00 | 4.187207                | 108.335695    |

**Step 2- Generate Python Code**

*System Prompt:* 
    
Given a user query and a pandas dataframe with the relevant data, generate syntatically correct python code wrapped in ```python {QUERY}`` that takes the raw dataframe and performs any computations to fully answer the user's query. Assume access to NumPy (v1.26.4), Pandas (v2.2.3) and that the dataframe is called df. The output of the code should be the variable that contains the result of the user's query.
*Labeled Answer:*
``` python

### calculate corr btwn 7yr tsy and stock closes 
corr = df['treasury_yield_7_year'].corr(df['stock_close'])
### output the variable that stores the result of the user's query
corr 
```
*Result of Executing Python:* 
-0.21388019833929192


In [3]:
def get_schema_context(config):
    db     = config.db
    tables = db.get_usable_table_names()

    schema_lines = []
    for table in tables:
        if table in ['ohlc', 'treasury_yields']:
            table_info = db.get_table_info([table])
            schema_lines.append(f"Table: {table}\n{table_info}\n")

    schema_context = (
    "DATABASE SCHEMA:\n" +
    "\n".join(schema_lines)
    )
    return schema_context

def extract_query(response: str, sql=True) -> str:
    pattern = r'```sql\n(.*?)\n```' if sql else r'```python\n(.*?)\n```'
    matches = re.findall(pattern, response, re.DOTALL)
    if matches:
        return matches[0].strip()
    
    print(f'Extracting Query Failed: returning response.strip(): \n {response.strip()}')
    ### if no matches
    return response.strip()

In [11]:
def execute_sql(user_query, config):
    """generate and execute sql"""
    schema_context = get_schema_context(config)
    sql_prompt     = f''' 
    Given a user query, generate a single syntatically correct SQLite query wrapped in ```sql QUERY``` that pulls all the 
    data relevant to the query. Do not perform any computation, assume that the user is able to take the raw data and 
    perform the relevant computations to reach the desired result. Be mindful of how many days of data to pull, as 
    certain queries may specific n days but require more than n to compute the result. For days do LIMIT rather than now().
    Make sure the column names in the resulting table are clear. Here is the database schema: {schema_context}
    Here is an example user query and then correct output:

    User Query: Calculate the correlation between 7 year treasury yields and stocks close over the last 30 days in the table.
    Correct Output: ``` sql 
    SELECT 
        y.date AS date,
        y.yield_7_year AS treasury_yield_7_year,
        o.close AS stock_close
    FROM 
        treasury_yields y
    INNER JOIN 
        ohlc o ON y.date = o.date
    ORDER BY 
        y.date DESC
    LIMIT 30;
    ```
    Notice there is no calculation calculation. This is the correct response, as 
    the correlation will be calculated later.
    
    Give the correct output for user query: {user_query}
    '''
    sql_query = extract_query(config.llm.invoke(sql_prompt).content)
    print(f'SQL Query: {sql_query}')
    df = pd.read_sql(sql_query, config.engine)
    df.head()
    return df 

def execute_python(user_query, df, config):
    """generate and execute python"""
    python_prompt = f''' 
    Given a user query and a pandas dataframe with the relevant data, generate syntatically correct python code 
    wrapped in ```python QUERY`` that takes the raw dataframe and performs any computations to fully answer the 
    user's query. Assume access to NumPy (v1.26.4), Pandas (v2.2.3) and that the dataframe is called df. The output 
    variable should always be called result. 

    Here is an example user query and df and then correct output:

    User Query: Calculate the correlation between 7 year treasury yields and stocks close over the last 30 days
    in the table.
    Dataframe (df): 
    Date,Treasury Yield (7-Year),Stock Close
    2024-01-01 00:00:00,4.113933,84.676268
    2023-12-29 00:00:00,4.117221,100.393128
    2023-12-28 00:00:00,2.391113,112.97598
    2023-12-27 00:00:00,1.482054,119.224503
    2023-12-26 00:00:00,4.187207,108.335695

    *Labeled Answer:*
    ``` python

    ### calculate corr btwn 7yr tsy and stock closes 
    result = df['treasury_yield_7_year'].corr(df['stock_close'])
    ```

    User Query: \n{user_query}
    df.head: \n{df.head()}
    '''

    code = extract_query(config.llm.invoke(python_prompt).content, sql=False)
    print(f'Python Code:\n {code}')
    
    namespace = {'pd': pd, 'np': np, 'df': df}
    exec(code, namespace)
    result = namespace.get('result', "No result variable found")
    print(f'Result:\n{result}')
    return result 

def run_agent(user_query, config=config):
    """main function to run agent"""
    print(f'User Query:\n {user_query}')
    start  = time.time()
    df     = execute_sql(user_query, config)
    result = execute_python(user_query, df, config)
    end    = time.time()

    print(f'Run Time: {round(end-start,2)}')
    return result

In [5]:
user_query = '''
Calculate the standard deviation of range := high of stocks - low of stocks over 30 days as well as the 
standard deviation of 5s10s := 10y tsy yield - 5y tsy yield over the same 30 days. Give me the absolute 
value between these 2 standard deviations. 
''' 
result = run_agent(user_query)

User Query:
 
Calculate the standard deviation of range := high of stocks - low of stocks over 30 days as well as the 
standard deviation of 5s10s := 10y tsy yield - 5y tsy yield over the same 30 days. Give me the absolute 
value between these 2 standard deviations. 

SQL Query: SELECT 
    o.date AS date,
    o.high - o.low AS stock_range,
    t.yield_10_year - t.yield_5_year AS treasury_5s10s
FROM 
    ohlc o
INNER JOIN 
    treasury_yields t ON o.date = t.date
ORDER BY 
    o.date DESC
LIMIT 30;
Python Code:
 # Calculate the standard deviation of stock_range and treasury_5s10s over 30 days
std_stock_range = df['stock_range'].std()
std_treasury_5s10s = df['treasury_5s10s'].std()

# Calculate the absolute difference between the two standard deviations
result = abs(std_stock_range - std_treasury_5s10s)
Result:
5.310255348485894
Run Time: 4.63


In [10]:
user_query = '''Find the days where stock price movement := close-open was more 
than 2 standard deviations from the mean'''
result     = run_agent(user_query)

User Query:
 Find the days where stock price movement := close-open was more 
than 2 standard deviations from the mean
SQL Query: SELECT 
    date,
    open,
    close
FROM 
    ohlc
WHERE 
    (close - open) > (SELECT AVG(close - open) FROM ohlc) + 2 * (SELECT (julianday('now') - julianday(date)) AS days_passed, 
    stddev(close - open) FROM ohlc)
OR 
    (close - open) < (SELECT AVG(close - open) FROM ohlc) - 2 * (SELECT (julianday('now') - julianday(date)) AS days_passed, 
    stddev(close - open) FROM ohlc);


OperationalError: (sqlite3.OperationalError) no such function: stddev
[SQL: SELECT 
    date,
    open,
    close
FROM 
    ohlc
WHERE 
    (close - open) > (SELECT AVG(close - open) FROM ohlc) + 2 * (SELECT (julianday('now') - julianday(date)) AS days_passed, 
    stddev(close - open) FROM ohlc)
OR 
    (close - open) < (SELECT AVG(close - open) FROM ohlc) - 2 * (SELECT (julianday('now') - julianday(date)) AS days_passed, 
    stddev(close - open) FROM ohlc);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)